El local Siempre Fresco se encuentra estudiando la provisión de harina para los próximos $T$ meses. A inicio de cada mes, la dueña del local puede determinar la cantidad de sacos de harina que pedirá para ese mes.

De acuerdo a reportes recabados, ha determinado que la demanda de sacos de harina es 1 con probabilidad 0,4 y 2 con probabilidad 0,6.

El ingreso por cada saco de harina vendido es de 10 dólares, el costo por cada saco de harina pedido es 5 dólares y, en caso de sobrar sacos de harina, de un periodo al siguiente, se almacenan para el mes siguiente a un costo de 3 dólares cada uno. Considere que no existe costo por demanda insatisfecha y que todos los sacos que se mantengan a inicio del periodo $T+1$ serán donados a una obra de caridad.

Considerando la situación descrita y que se poseen 0 sacos de harina en el local, se ha formulado modelo de programación dínamica

**Etapas**

$3$, una por cada mes.

**Variables de decisión**:

$X_n$: sacos a pedir el mes $n$. 

**Variables de estado**: 

$S_n$: cantidad de sacos de harina almacenados al inicio del mes $n$. 

**Reglas de transformación** 

$S_{n+1}=\max(0,S_n+X_n-D_n)$,

donde $D_n=\left\{\begin{array}{ccl}
	1& \text{ con probabilidad} & 0,4 \\
	2& \text{ con probabilidad} & 0,6 
\end{array}\right.$ 

**Función de recompensa**

$R_n(S_n,X_n,D_n) = 10\min(S_n+X_n,D_n)-5X_n-3\max(0,S_n+X_n-D_n)$

**Función recursiva**

$f_n(S_n,X_n)=\mathbb{E}(R_n(S_n,X_n,D_n)+f_{n+1}^*(S_{n+1}))$, para todo $n$.

**Función objetivo** 

$\displaystyle f_n^*(S_n)=\max_{X_n} (f_n(S_n,X_n))$ para todo $n$.

**Condiciones**

$f_{4}^*(S_4)=0$

$S_1=0$

**Restricciones**

$X_n, S_n \geq 0$, enteras, para todo $n$.

$X_n\leq 3$, para todo $n$.



In [3]:
def funcion_recompensa(S,X,D):
    R = 10*min(S+X,D)-5*X-3*max(0,S+X-D)
    return R

In [4]:
def funcion_recursiva(S,X,D):
    S2 = max(0,S+X-D)
    return S2

In [5]:
import numpy as np

In [11]:
T=3

ETAPAS=[]

for i in reversed(range(T+1)):
    tabla = -float("inf")*np.ones((2*T+1,3+1+2))
    for s in range(2*T+1):
        for x in range(3+1): #va del cero al tres
            if i == T:
                tabla[s,x]=0
            
            else:
                if x+s <= 2*T:
                    tabla[s,x]=0.4*(funcion_recompensa(s,x,1)+ETAPAS[0][funcion_recursiva(s,x,1),-1])+0.6*(funcion_recompensa(s,x,2)+ETAPAS[0][funcion_recursiva(s,x,2),-1])
                

            if tabla[s,x]>tabla[s,-1]:
                tabla[s,-1]=tabla[s,x]
                tabla[s,-2]=x

    ETAPAS.insert(0,tabla)

In [13]:
from tabulate import tabulate

In [15]:
for e, i in zip(range(T+1),ETAPAS):
    print(f'\nEtapa {e+1}\n', tabulate(i, headers='0 1 2 3 X Fx*'.split()))


Etapa 1
      0         1         2         3    X     Fx*
------  --------  --------  --------  ---  ------
11.8      16.8      18.6      15.6      2  18.6
21.8      23.6      20.6      16.368    1  23.6
28.6      25.6      21.368    13.992    0  28.6
30.6      26.368    18.992     7.8      0  30.6
31.368    23.992    12.8    -inf        0  31.368
28.992    17.8    -inf      -inf        0  28.992
22.8    -inf      -inf      -inf        0  22.8

Etapa 2
     0        1        2        3    X    Fx*
-----  -------  -------  -------  ---  -----
 5       10       11.8      8.72    2  11.8
15       16.8     13.72     7.4     1  16.8
21.8     18.72    12.4      1.4     0  21.8
23.72    17.4      6.4     -4.6     0  23.72
22.4     11.4      0.4   -inf       0  22.4
16.4      5.4   -inf     -inf       0  16.4
10.4   -inf     -inf     -inf       0  10.4

Etapa 3
    0       1       2       3    X    Fx*
----  ------  ------  ------  ---  -----
 0       5       4.8    -3.2    1    5
10       9

En este caso, la búsqueda de la solución es notoriamente más compleja que en el caso estudiado la semana pasada y por lo mismo se requeriría generar un nuevo programa para determinar la solución, ya que dependerá del valor que tome la variable aleatoria D y de esta forma se generará un árbol de decisión.

Notemos que igualmente podemos determinar la política óptima, tal cual como lo hacemos en el curso.

En este caso notamos que dada la estructura, conviene pedir siempre una unidad si es que partimos con 0, obteniendo una recompensa esperada de 18.6 dólares.

### Solución 2

En primer lugar necesitamos definir tres funciones: la función de transformación (función de actualización de la variable de estado), función de recompensa y la función recursiva.

Para las etapas usaremos un arreglo de tres dimensiones, la primera dimensión corresponderá a las distinas etapas, la segunda a los valores de la variable de estado y la tercera a los valores de la variable de decisión. Como en este caso es posible asignar cualquier cantidad de equipos médicos a un país (respetando las restricciones) las filas van de $0$ a $R$, al igual que las columnas.

Asignaremos el valor -1 para inicializar las matrices de las etapas, a fin de poder determinar aquellas celdas infactibles.

In [29]:
import numpy as np
def jardin(T): # T será la cantidad de días.

    etapas = -float("inf")*np.ones((T,2*T+1,4))
    mejores = np.zeros((2*T+1,T+1))
    
    for k in range(T-1,-1,-1):
        for i in range(2*T+1):
            for j in range(4):
                if i+j <= 2*T:
                    etapas[k,i,j]=0.4*(funcion_recompensa(i,j,1)+mejores[funcion_recursiva(i,j,1),k+1])+0.6*(funcion_recompensa(i,j,2)+mejores[funcion_recursiva(i,j,2),k+1])
            mejores[i,k]=np.max(etapas[k,i,:]) 
    return etapas,mejores

In [30]:
a,b = jardin(3)
print(a)
print(b)

[[[ 11.8    16.8    18.6    15.6  ]
  [ 21.8    23.6    20.6    16.368]
  [ 28.6    25.6    21.368  13.992]
  [ 30.6    26.368  18.992   7.8  ]
  [ 31.368  23.992  12.8      -inf]
  [ 28.992  17.8      -inf    -inf]
  [ 22.8      -inf    -inf    -inf]]

 [[  5.     10.     11.8     8.72 ]
  [ 15.     16.8    13.72    7.4  ]
  [ 21.8    18.72   12.4     1.4  ]
  [ 23.72   17.4     6.4    -4.6  ]
  [ 22.4    11.4     0.4      -inf]
  [ 16.4     5.4      -inf    -inf]
  [ 10.4      -inf    -inf    -inf]]

 [[  0.      5.      4.8    -3.2  ]
  [ 10.      9.8     1.8    -6.2  ]
  [ 14.8     6.8    -1.2    -9.2  ]
  [ 11.8     3.8    -4.2   -12.2  ]
  [  8.8     0.8    -7.2      -inf]
  [  5.8    -2.2      -inf    -inf]
  [  2.8      -inf    -inf    -inf]]]
[[18.6   11.8    5.     0.   ]
 [23.6   16.8   10.     0.   ]
 [28.6   21.8   14.8    0.   ]
 [30.6   23.72  11.8    0.   ]
 [31.368 22.4    8.8    0.   ]
 [28.992 16.4    5.8    0.   ]
 [22.8   10.4    2.8    0.   ]]
